#testing out the NY Times API

In [293]:
from urllib2 import Request, urlopen, URLError
import json

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [375]:
key = 'ec4e2a5e373e93f850f9be6c420bb73a:2:73271883'
isbn = '0553418025' #THE MARTIAN
#isbn = '1476783454' #the survivor
        
prefix = 'http://api.nytimes.com/svc/books/v3/lists/'
best_sellers = 'best-sellers/history.json?isbn='
api_key_string = "&api-key="  + key


URI = prefix + best_sellers + isbn + api_key_string 
print URI

http://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json?isbn=0553418025&api-key=ec4e2a5e373e93f850f9be6c420bb73a:2:73271883


In [376]:
request = Request(URI)
print request

<urllib2.Request instance at 0x104d3d170>


In [377]:
response = urlopen(request)
print response.read()

{"status":"OK","copyright":"Copyright (c) 2015 The New York Times Company.  All Rights Reserved.","num_results":1,"results":[{"title":"THE MARTIAN","description":"Abandoned by his crew, an astronaut embarks on a dogged quest to stay alive.","contributor":"by Andy Weir","author":"Andy Weir","contributor_note":"","price":0,"age_group":"","publisher":"Crown","isbns":[{"isbn10":"0804139024","isbn13":"9780804139021"},{"isbn10":"0553418025","isbn13":"9780553418026"},{"isbn10":"0091956145","isbn13":"9780091956141"},{"isbn10":"1594138540","isbn13":"9781594138546"},{"isbn10":"1101903589","isbn13":"9781101903582"},{"isbn10":"110190500X","isbn13":"9781101905005"}],"ranks_history":[{"primary_isbn10":"110190500X","primary_isbn13":"9781101905005","rank":1,"list_name":"Mass Market Paperback","display_name":"Paperback Mass-Market Fiction","published_date":"2015-10-25","bestsellers_date":"2015-10-10","weeks_on_list":8,"ranks_last_week":null,"asterisk":0,"dagger":0},{"primary_isbn10":"None","primary_isb

In [378]:
martians = urlopen(request)
partians = json.loads(martians.read())

In [379]:
type(partians)

dict

In [380]:
parsed_martian = partians

In [381]:
type(parsed_martian)

dict

In [382]:
parsed_martian.keys()

[u'status', u'num_results', u'results', u'copyright']

In [383]:
results = parsed_martian['results']
type(results[0])

dict

In [384]:
results[0].keys()

[u'publisher',
 u'description',
 u'author',
 u'price',
 u'title',
 u'ranks_history',
 u'reviews',
 u'isbns',
 u'contributor',
 u'contributor_note',
 u'age_group']

In [401]:
author = str(results[0]['author'])
title = str(results[0]['title'])
print title
print author

ranks_history = results[0]['ranks_history']

THE MARTIAN
Andy Weir


In [386]:
len(ranks_history)
ranks_history[0].keys()


[u'ranks_last_week',
 u'display_name',
 u'bestsellers_date',
 u'primary_isbn10',
 u'primary_isbn13',
 u'asterisk',
 u'rank',
 u'published_date',
 u'dagger',
 u'list_name',
 u'weeks_on_list']

In [402]:
entry_list = [None] * 20
counter = 0
for item in ranks_history:
    rank = item['rank']
    date = str(item['bestsellers_date'])
    weeks_on_list = item['weeks_on_list']
    book_list = str(item['list_name'])
    entry = [title, isbn, author, book_list, rank, date, weeks_on_list]
    entry_list[counter] = entry
    counter = counter + 1


In [443]:
import psycopg2

con = None

con = psycopg2.connect(database='nytimes') 
cur = con.cursor()
cur.execute("CREATE TABLE books(title VARCHAR(255), isbn_10 INT, author VARCHAR(255), list VARCHAR(255), rank INT, date VARCHAR(255), weeks_on_list INT, description VARCHAR(255))")

con.commit()
con.close()

In [442]:
con = None

con = psycopg2.connect(database='nytimes') 
cur = con.cursor()

query = "INSERT INTO books (title, isbn_10, author, list, rank, date, weeks_on_list) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur.executemany(query, entry_list)

con.commit()
con.close()